# Optimizers - When, Where and How to Tweak Them

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
from exp.nb_08 import *

RuntimeError: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero.

## Getting Imagenette Data From the DataBlock NB

In [ ]:
path = datasets.untar_data(datasets.URLs.IMAGENETTE_160)

In [ ]:
tfms = [make_rgb, ResizeFixed(128), to_byte_tensor, 
        to_float_tensor]
bs = 128

img_list = ImageList.from_files(path, tfms=tfms)
split_data = SplitData.split_by_func(img_list, 
                                     partial(grandparent_splitter,
                                            valid_name='val'))

labels = label_by_func(split_data, parent_labeler, 
                       proc_y=CategoryProcessor())
data = labels.to_databunch(bs, c_in=3, c_out=10, 
                           num_workers=4)

In [ ]:
# Create a model
nfs = [32, 64, 128, 256]

In [ ]:
callbacks = [partial(AvgStatsCallback, accuracy),
             CudaCallback,
             partial(BatchTransformXCallback, norm_imagenette)]

In [ ]:
# Baseline training with vanilla SGD
learn, run = get_learn_run(nfs, data, lr=0.4,
                          layer=conv_layer, cbs=callbacks)

In [ ]:
run.fit(1, learn)

## Refining the Optimizer

**`NOTES`**

- The base PyTorch optimizer in `torch.optim` is a dictionary which stores the hyper-parameters and references to the parameters of the model we want to train in parameter groups.

- It contains `step` which updates our parameters with gradients and a method `zero_grad` to detach and zero the gradients of our parameters.

- We will build a more flexible equivalent from scratch. Here, the step function loops over all the parameters to execute the step using stepper functions, which we will provide when initializing the optimizer.

- This will end up giving us parameter groups / layer groups.

In [ ]:
class Optimizer():
    def __init__(self, params, steppers, **defaults):
        # Could be a generator
        self.param_groups = list(params)
        # Ensuring params is a list of lists of parameter tensors
        if not isinstance(self.param_groups[0], list):
            self.param_groups = [self.param_groups]
        # Creating a dictionary for individual param groups with
        # their own references
        self.hypers = [{**defaults} for p in self.param_groups]
        self.steppers = listify(steppers)
        
    def grad_params(self):
        return [(p, hyper) for pg, hyper in zip(self.param_groups, self.hypers)
                for p in pg if p.grad is not None]
    
    def zero_grad(self):
        for p, hyper in self.grad_params():
            p.grad.detach_()
            p.grad.zero_()
            
    def step(self): 
        # This step function doesn't do anything except carry out a
        # composition on items we pass on, which in turn carry out
        # their own operations. For e.g. one cycle annealing, 
        # discriminative LRs etc.
        for p, hyper in self.grad_params():
            compose(p, self.steppers, **hyper)

In [ ]:
# To carry out SGD, via the stepper
def sgd_step(param, lr, **kwargs):
    param.data.add_(-lr, param.grad.data)
    return param

In [ ]:
opt_func = partial(Optimizer, steppers=[sgd_step])

**`NOTES`**

- After changing the optimizer, we will need to adjust the callbacks which used the properties from the PyTorch optimizer.

- Hyper-parameters are in the list of dictionaries `opt.hypers`.

In [ ]:
# Updating the Recorder, ParamScheduler and LR_Find classes
class Recorder(Callback):
    def begin_fit(self): self.lrs, self.losses = [], []
        
    def after_batch(self):
        if not self.in_train: 
            return
        self.lrs.append(self.opt.hypers[-1]['lr'])
        self.losses.append(self.loss.detach().cpu())
        
    def plot_lr(self): 
        plt.plot(self.lrs)
        
    def plot_loss(self):
        plt.plot(self.losses)
        
    def plot(self, skip_last=0):
        losses = [o.item() for o in self.losses]
        n = len(losses) - skip_last
        plt.xscale('log')
        plt.plot(self.lrs[:n], losses[:n])
        

class ParamScheduler(Callback):
    _order = 1
    
    def __init__(self, pname, sched_funcs):
        self.pname, self.sched_funcs = pname, listify(sched_funcs)
    
    def begin_batch(self):
        if not self.in_train:
            return
        fs = self.sched_funcs
        if len(fs)==1: 
            fs = fs*len(self.opt.param_groups)
        pos = self.n_epochs / self.epochs
        for f, h in zip(fs, self.opt.hypers):
            h[self.pname] = f(pos)
                

class LR_Find(Callback):
    _order = 1
    def __init(self, max_iter=100, min_lr=1e-6, max_lr=10):
        self.max_iter, self.min_lr, self.max_lr = max_iter, min_lr, max_lr
        self.best_loss = 1e9
        
    def begin_batch(self):
        if not self.in_train: 
            return
        pos = self.n_iter / self.max_iter
        lr = self.min_lr * (self.max_lr / self.min_lr) ** pos
        for pg in self.opt.hypers:
            pg['lr'] = lr
            
    def after_step(self):
        if self.n_iter >= self.max_iter or self.loss > self.best_loss*10:
            raise CancelTrainException()
        if self.loss < self.best_loss:
            self.best_loss = self.loss
        

In [ ]:
# Checking to see if the param scheduler and the recorder are working properly
sched = combine_scheds([0.3, 0.7], [sched_cos(0.3, 0.6), sched_cos(0.6, 0.2)])

callbacks = [partial(AvgStatsCallback, accuracy),
             CudaCallback, Recorder, 
             partial(ParamScheduler, 'lr', sched)]

In [ ]:
learn, run = get_learn_run(nfs, data, lr=0.4, layer=conv_layer,
                           cbs=callbacks, opt_func=opt_func)

In [ ]:
%time run.fit(1, learn)

In [ ]:
# Plotting loss
run.recorder.plot_loss()

In [ ]:
# Plotting LR
run.recorder.plot_lr()

## Implementing Weight Decay

**`NOTES`**

- In order to avoid over-fitting we will implement L2 regularization which is commonly known as weigth decay. Here we will be adding the sum of all the weights squared to our loss function. Doing this will ensure that the weights remain as small as possible when we compute the gradients.

- Limiting the weights from growing at a high rate is going to hinder the training of the model, but it yeilds a state where it generalizes better.

- Weight decay is a parameter that controls the sum of squares we add to our loss:
```python
loss_with_wd = loss + (wd/2) * (weights**2).sum()
```

- In practice it would be very inefficient to compute that big sum and add it to the loss. Alternatively, adding the big sum to our loss is the same as doing:
```python
weight.grad += wd * weight
```

- For every weight in our vanilla SGD is equivalent to updating the parameters with:
```python
wigh5t = weight - lr * (weight.grad + wd*weight)
```

- This results in the "decay" of each weight by a factor `lr * wd`. However, this only works for standard SGD, as we have seen with momentum, RMSProp and Adam, the update has some additional formulas around the gradient. In those cases the formula that comes from L2 regularization is:
```python
weight.grad += wd * weight
```
becomes different than weight decay
```python
new_weight = weight - lr * weight.grad - lr * wd * weight
```

- It should be noted that most libraries use the first formula, but as pointed out in the paper [Decoupled Weight Regularization](https://arxiv.org/pdf/1711.05101.pdf), it is better to use the second one with the Adam optimizer.


In [ ]:
def weight_decay(param, lr, wd, **kwargs):
    param.data.mul_(1 - lr*wd)
    return param

weight_decay._defaults = dict(wd=0.)

In [ ]:
# L2 regularization adds wd*weight to the gradients
def l2_regularization(param, lr, wd, **kwargs):
    param.grad.data.add_(wd, param.data)
    return param

l2_regularization._defaults = dict(wd=0.)

In [ ]:
# Allowing the steppers to add to our default hyper param values
# The helper function adds in 'dest' the key/values it finds while going
# through `os` and applying `f` when there was no `key` of the same name
def maybe_update(os, dest, f):
    for o in os:
        for k,v in f(o).items():
            if k not in dest:
                dest[k] = v
                
def get_defaults(d):
    return getattr(d, '_defaults', {})

 - The optimizer remains mostly the same as before, we only take the default values of the steppers when none are provided in the kwargs.

In [ ]:
#export
class Optimizer():
    def __init__(self, params, steppers, **defaults):
        self.steppers = listify(steppers)
        maybe_update(self.steppers, defaults, get_defaults)
        # might be a generator
        self.param_groups = list(params)
        # ensure params is a list of lists
        if not isinstance(self.param_groups[0], list): 
            self.param_groups = [self.param_groups]
        self.hypers = [{**defaults} for p in self.param_groups]

    def grad_params(self):
        return [(p,hyper) for pg,hyper in zip(self.param_groups,self.hypers)
            for p in pg if p.grad is not None]

    def zero_grad(self):
        for p,hyper in self.grad_params():
            p.grad.detach_()
            p.grad.zero_()

    def step(self):
        for p,hyper in self.grad_params(): 
            compose(p, self.steppers, **hyper)

In [ ]:
sgd_opt = partial(Optimizer, steppers=[weight_decay, sgd_step])

In [ ]:
learn, run = get_learn_run(nfs, data, lr=0.4, layer=conv_layer,
                           cbs=callbacks, opt_func=sgd_opt)

In [ ]:
# Before training let's check whether everythign works as intended
# If we dont't provide a value for 'wd', we will pull the corresponding
# default from 'weight_decay'
model = learn.model

In [ ]:
opt = sgd_opt(model.parameters(), lr=0.1)
test_eq(opt.hypers[0]['wd'], 0.)
test_eq(opt.hypers[0]['lr'], 0.1)

In [ ]:
# Now, when we provide a value, it overrides the default
opt = sgd_opt(model.parameters(), lr=0.1, wd=1e-4)
test_eq(opt.hypers[0]['wd'], 0.0001)
test_eq(opt.hypers[0]['lr'], 0.1)

In [ ]:
# Applying what we have built to our model
cbfs = [partial(AvgStatsCallback, accuracy), CudaCallback]

In [ ]:
learn, run = get_learn_run(nfs, data, 0.3, layer=conv_layer,
                          cbs=callbacks, opt_func=partial(sgd_opt, wd=0.01))

In [ ]:
%time run.fit(1, learn)

In [ ]:
run.recorder.plot_loss()

In [ ]:
run.recorder.plot_lr()

**We've already improved the baseline!!**

## Adding Momentum to the Optimizer

**`NOTES`**

- The key factor in adding Momentum to the optimzer is adding some state.

- Moving averages of the gradients need to be saved and stored inside the optimizer state.

- To do this, we will be adding statistics. These are objects with two methods i.e. `init_state` which returns the initial state (a tensor of 0. for the moving average of gradients) and `update`, which updates the state with the new gradient value.

- `_defaults` will also be read to provide default values to hyper-parameters.

In [ ]:
class StatefulOptimizer(Optimizer):
    def __init__(self, params, steppers, stats=None, **defaults):
        self.stats = listify(stats)
        maybe_update(self.stats, defaults, get_defaults)
        super().__init__(params, steppers, **defaults)
        self.state = {}
        
    def step(self):
        for param, hyper in self.grad_params():
            if param not in self.state:
                # Create a state for param and call the statistics to initialize it
                self.state[param] = {}
                maybe_update(self.stats, self.state[param],
                             lambda o: o.init_state(param))
            state = self.state[param]
            for stat in self.stats:
                state = stat.update(param, state, **hyper)
            compose(param, self.steppers, **state, **hyper)
            self.state[param] = state

In [ ]:
# Stats, at least the way they work and are updated, resemble steppers 
class Stat():
    _defaults = {}
    def init_state(self, param):
        raise NotImplementedError
    
    def update(self, param, state, **kwargs):
        raise NotImplementedError

In [ ]:
# We will use momentum as an example of adding Stats
class AverageGrad(Stat):
    _defaults = dict(mom=0.9)
    
    def init_state(self, param):
        return {'grad_avg': torch.zeros_like(param.grad.data)}
    
    def update(self, param, state, mom, **kwargs):
        state['grad_avg'].mul_(mom).add_(param.grad.data) #this is momentum as per the definition
        return state

In [ ]:
# Creating the momentum step.
def momentum_step(param, lr, grad_avg, **kwargs):
    param.data.add_(-lr, grad_avg)
    return param

In [ ]:
sgd_mom_opt = partial(StatefulOptimizer, steppers=[momentum_step, weight_decay],
                      stats=AverageGrad(), wd=0.01)

In [ ]:
learn, run = get_learn_run(nfs, data, lr=0.3, layer=conv_layer,
                          cbs=callbacks, opt_func=sgd_mom_opt)

In [ ]:
run.fit(1, learn)

### Quick Momentum Experiments

- Here, we will try to figure out what momentum actually does to the gradients by generating some plots.

In [ ]:
x = torch.linspace(-4, 4, 200)
y = torch.randn(200) + 0.3
betas = [0.5, 0.7, 0.9, 0.999]

In [ ]:
def plot_momentum(f):
    _, axs = plt.subplots(2, 2, figsize=(12,8))
    for beta, ax in zip(betas, axs.flatten()):
        ax.plot(y, linestyle='None', marker='.')
        avg, res = None, []
        for i, yi in enumerate(y):
            avg, param = f(avg, beta, yi, i)
            res.append(param)
        ax.plot(res, color='red')
        ax.set_title(f'beta={beta}')

In [ ]:
# Using regular momentum
def mom1(avg, beta, yi, i):
    if avg is None:
        avg = yi
    res = beta*avg + yi
    return res, res

plot_momentum(mom1)

In [ ]:
# Regular momentum can easily cause the results to diverge, especially if the value is
# too high.
# A better way to smooth noisy data is to do an exponentially weighted moving average.
# Here there is a dampening of (1-beta) in front of the new value. We will define
# lin_comb(linear combination) to make this easier.
def lin_comb(v1, v2, beta):
    return beta*v1 + (1-beta)*v2 

In [ ]:
def momentum2(avg, beta, yi, i):
    if avg is None: 
        avg=yi
    avg = lin_comb(avg, yi, beta)
    return avg, avg

plot_momentum(momentum2)

 - This is the result when our data is purely random. If it has a certain shape, then it will get that shape, but with a delay for high beta.

In [ ]:
y = 1 - (x/3)**2 + torch.randn(200)*0.1

In [ ]:
y[0] = 0.5

In [ ]:
plot_momentum(momentum2)

- In the case of beta=0.999, item 1 is massively biasing the start of optimizing.

- We will now introduce debiasing. Basically, we will be dividing our moving average with sum of the coefficients:
$\begin{align*}
S = 1 - \beta^{i+1}
\end{align*}$

- By dividing by this term, we make our moving average a true average (all coefficients we used for the average sum up to 1).


In [ ]:
def momentum3(avg, beta, yi, i):
    if avg is None:
        avg = 0
    avg = lin_comb(avg, yi, beta)
    return avg, avg / (1-beta**(i+1))

plot_momentum(momentum3)

## Introducing Adam

- With Adam, we use the gradient averages with dampening which won't be the same as SGD with momentum.

In [ ]:
# Making the necessary changes to AverageGrad
class AverageGrad(Stat):
    _defaults = dict(mom=0.9)
    
    def __init__(self, dampening: bool=False):
        self.dampening=dampening
    
    def init_state(self, param):
        return {'grad_avg': torch.zeros_like(param.grad.data)}
    
    def update(self, param, state, mom, **kwargs):
        state['mom_damp'] = 1-mom if self.dampening else 1.
        state['grad_avg'].mul_(mom).add_(state['mom_damp'], param.grad.data)
        return state

In [ ]:
# We will also need to track the moving average of the gradients squared.
class AverageSqrGrad(Stat):
    _defaults = dict(sqr_mom=0.99)
    
    def __init__(self, dampening:bool=True):
        self.dampening = dampening
        
    def init_state(self, param):
        return {'sqr_avg': torch.zeros_like(param.grad.data)}
    
    def update(self, param, state, sqr_mom, **kwargs):
        state['sqr_damp'] = 1-sqr_mom if self.dampening else 1.
        state['sqr_avg'].mul_(sqr_mom).addcmul_(state['sqr_damp'], 
                                                param.grad.data, param.grad.data)
        return state

In [ ]:
# We will also need the number of steps done during training for debiasing.
class StepCount(Stat):
    def init_state(self, param):
        return {'step': 0}
    
    def update(self, param, state, **kwargs):
        state['step'] += 1
        return state

In [ ]:
# This helper function computes the debias term. If we're dampening then 'damp = 1-mom'
# while if we're not dampening `damp=1`,we will need to divide by '1-mom' since that
# term is missing in every calculation.
def debias(mom, damp, step):
    return damp * (1 - mom**step) / (1 - mom)

In [ ]:
# This is the Adam step
def adam_step(param, lr, mom, mom_damp, step, sqr_mom, sqr_damp,
              grad_avg, sqr_avg, eps, **kwargs):
    debias1 = debias(mom, mom_damp, step)
    debias2 = debias(sqr_mom, sqr_damp, step)
    param.data.addcdiv_(-lr / debias1, grad_avg, (sqr_avg/debias2).sqrt() + eps) # eps inside or outside the sqrt()??
    return param

adam_step._defaults = dict(eps=1e-5)

In [ ]:
def adam_opt(extra_step=None, **kwargs):
    return partial(StatefulOptimizer, 
                   steppers=[adam_step, weight_decay]+listify(extra_step),
                   stats=[AverageGrad(dampening=True), AverageSqrGrad(), StepCount()],
                   **kwargs)

In [ ]:
learn, run = get_learn_run(nfs, data, lr=0.001, layer=conv_layer, cbs=callbacks,
                          opt_func=adam_opt())

In [ ]:
run.fit(5, learn)

In [ ]:
run.recorder.plot_loss()

In [ ]:
run.recorder.plot_lr()

## Introducing the LAMB Optimizer